**Support code for Exploratory Report** <br>
https://datastudio.google.com/u/0/reporting/f8bd5e2a-589c-4344-9cfc-36ef1fa471b4

In [1]:
import logging
import pandas as pd
import numpy as np
import pandas_gbq
from datetime import datetime
from web3 import Web3
from google.oauth2 import service_account

from uniswap_utils import getPoolPrice #https://github.com/amantay-a/misc/blob/master/uniswap_utils.py

token_dict = {'ETH':'0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', #WETH
              'BTC':'0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599', #WBTC
              'USDC':'0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
               'DAI':'0x6B175474E89094C44Da98b954EedeAC495271d0F',
             '1INCH':'0x111111111117dC0aa78b770fA6A738034120C302',
              'AAVE':'0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9',
              'COMP':'0xc00e94Cb662C3520282E6f5717214004A7f26888',
               'DPI':'0x1494CA1F11D487c2bBe4543E90080AeBa4BA3C2b',
               'FEI':'0x956F47F50A910163D8BF957Cf5846D573E7f87CA',
              'LINK':'0x514910771AF9Ca656af840dff83E8264EcF986CA',
               'SNX':'0xC011a73ee8576Fb46F5E1c5751cA3B9Fe0af2a6F',
               'UNI':'0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984',
               'YFI':'0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e',
              'LUNA':'0xd2877702675e6cEb975b4A1dFf9fb7BAF4C91ea9', #WLUNA
               'FTM':'0x4E15361FD6b4BB609Fa63C81A2be19d873717870',
             'SUSHI':'0x6B3595068778DD592e39A122f4f5a5cF09C90fE2',
               'CRV':'0xD533a949740bb3306d119CC777fa900bA034cd52',
               'CVX':'0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B',
               'LDO':'0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32',
             'STETH':'0xDFe66B14D37C77F4E9b180cEb433d1b164f0281D',
              'FRAX':'0x853d955aCEf822Db058eb8505911ED77F175b99e',
              'LUSD':'0x5f98805A4E8be255a32880FDeC7F6728C6568bA0',
              'SUSD':'0x57Ab1ec28D129707052df4dF418D58a2D46d5f51',
              'GUSD':'0x056Fd409E1d7A124BD7017459dFEa2F387b6d5Cd',
              'USDT':'0xdAC17F958D2ee523a2206206994597C13D831ec7'
            }
# To be able to call historical prices, RPC must support archive mode, suggest using alchemy as it provide archive rpc for free
RPCEndpoint = ''
assert RPCEndpoint, "Set up Archive Endpoint"

w3 = Web3(Web3.HTTPProvider(RPCEndpoint, request_kwargs={'timeout': 20}))
print(f'Blockchain connected: {w3.isConnected()}')

Blockchain connected: True


In [2]:
twapwindows =  [1, 60*10, 60*20]
poolfee = [3000, 5000, 10000]

def get_uni_price(w3, tokenA, tokenB, block, TWAPWindows, PoolFees):
    
    price_twap = getPoolPrice(w3, 
                              tokenA, 
                              tokenB,
                              TWAPWindows = twapwindows,
                              blocks = [block],
                              PoolFees = poolfee
                             )
    
    #print(price_twap)
    prices = []    
    ret = {}
    for w in TWAPWindows:
        for i, f in enumerate(PoolFees):
            price = price_twap[block][i]['twap'][w]
            #print('price', price)
            if price:
                prices.append(price)    
        if prices:
            median_price = np.percentile(prices,50, interpolation ='lower')
            ret.update({'twap_'+str(w):median_price}) 
        else:
            ret.update({'twap_'+str(w):None})
    return ret, price_twap

In [3]:
credentials = service_account.Credentials.from_service_account_file(
    'gearbox-336415-5ed144668529.json',
)
gcp_project_id = 'gearbox-336415'

bq_select = '''with u as (select ticker, max(blockNumber) as twap_max_block from gearbox.uniswap_prices group by ticker) 
               select 
                   cl.ticker, cl.blockNumber, cl.updated_at, cl.price_decimal as price_chainlink
               from 
                   gearbox.oracle_price_history cl
                   left join u on u.ticker = cl.ticker
               where cl.base !='usd'
                   and cl.blockNumber > coalesce(twap_max_block,0)
               order by cl.ticker, cl.blockNumber

            '''
df_cl = pandas_gbq.read_gbq(bq_select, 
                             project_id=gcp_project_id,
                             progress_bar_type = None,)
display(df_cl)


,ticker,blockNumber,updated_at,price_chainlink
0,aave-eth,12069464.0,2021-03-19 13:29:36+00:00,0.205292
1,aave-eth,12073371.0,2021-03-20 03:55:27+00:00,0.209424
2,aave-eth,12076850.0,2021-03-20 16:52:24+00:00,0.213676
3,aave-eth,12077634.0,2021-03-20 19:36:42+00:00,0.209280
4,aave-eth,12078449.0,2021-03-20 22:36:23+00:00,0.205071
...,...,...,...,...
56509,yfi-eth,14665953.0,2022-04-27 10:30:13+00:00,6.124033
56510,yfi-eth,14666959.0,2022-04-27 14:18:14+00:00,6.196324
56511,yfi-eth,14667730.0,2022-04-27 16:57:02+00:00,6.133839
56512,yfi-eth,14668721.0,2022-04-27 20:45:54+00:00,6.196438


In [4]:
df = df_cl
df['tokenA'] = df['ticker'].apply(lambda x: token_dict[x.split('-')[0].upper()])
df['tokenB'] = df['ticker'].apply(lambda x: token_dict[x.split('-')[1].upper()])
df

,ticker,blockNumber,updated_at,price_chainlink,tokenA,tokenB
0,aave-eth,12069464.0,2021-03-19 13:29:36+00:00,0.205292,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
1,aave-eth,12073371.0,2021-03-20 03:55:27+00:00,0.209424,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
2,aave-eth,12076850.0,2021-03-20 16:52:24+00:00,0.213676,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
3,aave-eth,12077634.0,2021-03-20 19:36:42+00:00,0.209280,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
4,aave-eth,12078449.0,2021-03-20 22:36:23+00:00,0.205071,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
...,...,...,...,...,...,...
56509,yfi-eth,14665953.0,2022-04-27 10:30:13+00:00,6.124033,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
56510,yfi-eth,14666959.0,2022-04-27 14:18:14+00:00,6.196324,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
56511,yfi-eth,14667730.0,2022-04-27 16:57:02+00:00,6.133839,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
56512,yfi-eth,14668721.0,2022-04-27 20:45:54+00:00,6.196438,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2


In [5]:
twapwindow_cols = ['twap_'+str(x) for x in twapwindows]
df[twapwindow_cols] = None
cnt = 0
print(datetime.utcnow(),'Start')
for ticker in df['ticker'].unique():
    df_ticker = df[df['ticker']==ticker].copy()
    print(datetime.utcnow(),f'{cnt}/{len(df)} ({ticker})..')
    for x in df_ticker.itertuples():
        prices, raw = get_uni_price(w3, x.tokenA, x.tokenB, int(x.blockNumber), twapwindows, poolfee)
        prices = list(prices.values())
        df_ticker.loc[(df['blockNumber'] == x.blockNumber)&(df['ticker'] == x.ticker), twapwindow_cols+['raw']] = prices + [str(raw)]
        cnt+=1
        if cnt%100==0:
            print(datetime.utcnow(),f'{cnt}/{len(df)} ({ticker})..')
    df_ticker[twapwindow_cols] = df_ticker[twapwindow_cols].astype('float64')
    pandas_gbq.to_gbq(df_ticker, 
                      'gearbox.uniswap_prices',
                      project_id=gcp_project_id,
                      if_exists = 'append', #replace,
                      progress_bar = False)
display(df)

2022-04-28 10:01:43.555571 Start
2022-04-28 10:01:43.584701 0/56514 (aave-eth)..
2022-04-28 10:03:36.081965 100/56514 (aave-eth)..
2022-04-28 10:05:24.603038 200/56514 (aave-eth)..
2022-04-28 10:06:52.762094 300/56514 (aave-eth)..
2022-04-28 10:08:19.014250 400/56514 (aave-eth)..
2022-04-28 10:09:30.261347 500/56514 (aave-eth)..
2022-04-28 10:10:35.580361 600/56514 (aave-eth)..
2022-04-28 10:11:45.578351 700/56514 (aave-eth)..
2022-04-28 10:12:53.112159 800/56514 (aave-eth)..
2022-04-28 10:13:56.720822 900/56514 (aave-eth)..
2022-04-28 10:14:59.810577 1000/56514 (aave-eth)..
2022-04-28 10:16:01.585173 1100/56514 (aave-eth)..
2022-04-28 10:17:00.764935 1200/56514 (aave-eth)..
2022-04-28 10:17:59.471057 1300/56514 (aave-eth)..
2022-04-28 10:18:22.526078 1332/56514 (btc-eth)..
2022-04-28 10:19:41.307477 1400/56514 (btc-eth)..
2022-04-28 10:21:06.946292 1500/56514 (btc-eth)..
2022-04-28 10:22:26.057003 1600/56514 (btc-eth)..
2022-04-28 10:23:44.326433 1700/56514 (btc-eth)..
2022-04-28 10:2

2022-04-28 13:26:52.767741 15400/56514 (frax-eth)..
2022-04-28 13:27:50.228969 15500/56514 (frax-eth)..
2022-04-28 13:28:48.301742 15600/56514 (frax-eth)..
2022-04-28 13:29:45.045671 15700/56514 (frax-eth)..
2022-04-28 13:30:40.653390 15800/56514 (frax-eth)..
2022-04-28 13:31:36.444196 15900/56514 (frax-eth)..
2022-04-28 13:32:37.619515 16000/56514 (frax-eth)..
2022-04-28 13:32:42.554654 16000/56514 (ftm-eth)..
2022-04-28 13:34:39.518592 16100/56514 (ftm-eth)..
2022-04-28 13:36:29.721291 16200/56514 (ftm-eth)..
2022-04-28 13:38:19.209297 16300/56514 (ftm-eth)..
2022-04-28 13:40:10.504225 16400/56514 (ftm-eth)..
2022-04-28 13:42:01.225723 16500/56514 (ftm-eth)..
2022-04-28 13:43:53.692572 16600/56514 (ftm-eth)..
2022-04-28 13:45:42.881610 16700/56514 (ftm-eth)..
2022-04-28 13:47:32.099688 16800/56514 (ftm-eth)..
2022-04-28 13:49:01.321583 16900/56514 (ftm-eth)..
2022-04-28 13:50:28.431529 17000/56514 (ftm-eth)..
2022-04-28 13:51:38.976783 17100/56514 (ftm-eth)..
2022-04-28 13:52:45.8581

2022-04-28 16:37:09.525380 30600/56514 (snx-eth)..
2022-04-28 16:38:58.830400 30700/56514 (snx-eth)..
2022-04-28 16:40:40.201067 30800/56514 (snx-eth)..
2022-04-28 16:42:04.048533 30900/56514 (snx-eth)..
2022-04-28 16:43:29.305655 31000/56514 (snx-eth)..
2022-04-28 16:44:54.576990 31100/56514 (snx-eth)..
2022-04-28 16:45:58.802846 31200/56514 (snx-eth)..
2022-04-28 16:47:01.337347 31300/56514 (snx-eth)..
2022-04-28 16:48:07.746386 31400/56514 (snx-eth)..
2022-04-28 16:49:14.957111 31500/56514 (snx-eth)..
2022-04-28 16:50:23.065140 31600/56514 (snx-eth)..
2022-04-28 16:51:26.972707 31700/56514 (snx-eth)..
2022-04-28 16:52:29.527617 31800/56514 (snx-eth)..
2022-04-28 16:53:29.518212 31900/56514 (snx-eth)..
2022-04-28 16:54:27.901455 32000/56514 (snx-eth)..
2022-04-28 16:55:28.358538 32100/56514 (snx-eth)..
2022-04-28 16:56:29.612610 32200/56514 (snx-eth)..
2022-04-28 16:57:30.198949 32300/56514 (snx-eth)..
2022-04-28 16:58:30.268460 32400/56514 (snx-eth)..
2022-04-28 16:59:06.984126 3244

2022-04-28 20:03:39.969324 45900/56514 (usdc-eth)..
2022-04-28 20:04:49.125417 46000/56514 (usdc-eth)..
2022-04-28 20:05:56.677615 46100/56514 (usdc-eth)..
2022-04-28 20:07:03.448509 46200/56514 (usdc-eth)..
2022-04-28 20:08:10.228663 46300/56514 (usdc-eth)..
2022-04-28 20:09:15.454626 46400/56514 (usdc-eth)..
2022-04-28 20:10:21.048010 46500/56514 (usdc-eth)..
2022-04-28 20:11:23.926072 46600/56514 (usdc-eth)..
2022-04-28 20:12:25.245027 46700/56514 (usdc-eth)..
2022-04-28 20:13:04.108069 46752/56514 (usdt-eth)..
2022-04-28 20:13:58.012572 46800/56514 (usdt-eth)..
2022-04-28 20:15:47.463339 46900/56514 (usdt-eth)..
2022-04-28 20:17:35.833229 47000/56514 (usdt-eth)..
2022-04-28 20:19:24.820811 47100/56514 (usdt-eth)..
2022-04-28 20:21:13.861980 47200/56514 (usdt-eth)..
2022-04-28 20:23:04.403433 47300/56514 (usdt-eth)..
2022-04-28 20:24:54.019982 47400/56514 (usdt-eth)..
2022-04-28 20:26:44.222054 47500/56514 (usdt-eth)..
2022-04-28 20:28:13.528839 47600/56514 (usdt-eth)..
2022-04-28 2

,ticker,blockNumber,updated_at,price_chainlink,tokenA,tokenB,twap_1,twap_600,twap_1200
0,aave-eth,12069464.0,2021-03-19 13:29:36+00:00,0.205292,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
1,aave-eth,12073371.0,2021-03-20 03:55:27+00:00,0.209424,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
2,aave-eth,12076850.0,2021-03-20 16:52:24+00:00,0.213676,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
3,aave-eth,12077634.0,2021-03-20 19:36:42+00:00,0.209280,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
4,aave-eth,12078449.0,2021-03-20 22:36:23+00:00,0.205071,0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
...,...,...,...,...,...,...,...,...,...
56509,yfi-eth,14665953.0,2022-04-27 10:30:13+00:00,6.124033,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
56510,yfi-eth,14666959.0,2022-04-27 14:18:14+00:00,6.196324,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
56511,yfi-eth,14667730.0,2022-04-27 16:57:02+00:00,6.133839,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None
56512,yfi-eth,14668721.0,2022-04-27 20:45:54+00:00,6.196438,0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,None,None,None


# Exploratory Report
https://datastudio.google.com/u/0/reporting/f8bd5e2a-589c-4344-9cfc-36ef1fa471b4